## 测试分组回测（group_trader）

In [2]:
from billions.data_loader import *
from billions.back_trader import *

def test(name, peride):
    # read data
    fname = 'data/' + name + '.csv'; cname = 'data/close.csv'; ftype = 'csv'
    data = loader_factor_close(fname, cname, ftype)

    # run trade
    gt = group_trader(data, number=5)
    gt.run(peride=peride)
    gt.plot(name)

for name in ['betastd', 'corrstd', 'r2']:
    for peride in ['D', 'M', 20]:
        test(name, peride)

ModuleNotFoundError: No module named 'billions'

## 测试海龟

In [ ]:
from billions.data_loader import *
from billions.back_trader import *

class turtle(trader):
    def __init__(self, data, total_amount=1000000, peride='D'):
        super().__init__(data, total_amount=total_amount, peride=peride)
        self.codes = np.unique(np.array(list(data.index))[:, 1])

        self.data = data

    def trade(self, date, prices):
        for code in self.codes:
            today = self.data.loc[date, code]

            if today.close > today.up:
                self.profolio.buy(code, 100, prices[code])

            if today.close < today.down:
                if code in self.profolio.profolio.keys():
                    self.profolio.sell(code, self.profolio.profolio[code]['amount'], prices[code])
                # self.profolio.sell(code, today.close*100)


In [ ]:
data = load_price('data/600000.csv')
data['up'] = data['high'].shift().rolling(20).apply(np.max, raw=False)
data['down'] = data['low'].shift().rolling(20).apply(np.min, raw=False)
data['pre_close'] = data['close'].shift()
data = data[20:]

t = turtle(data)
t.run()
p = t.ploter()
p.plot()
